In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re

In [ ]:
# Load dataset
def load_data():
    data = pd.read_csv('#PPKM.csv')#ubah nama file sesai dengan nama file kalian
    return data

tweet_df = load_data()
tweet_df.head()

#definisi dataframe
df  = pd.DataFrame(tweet_df[['Kalimat', 'label']])#ubah dataframe sesuai column kalian

In [ ]:
df.head()

,Kalimat,label
0,Ganjar Setuju Vaksinasi jadi Indikator PPKM Me...,Netral
1,Live now on YouTube @ Sahabat Jannah !! Love N...,Netral
2,Dewan Perwakilan Rakyat Daerah (DPRD) Kota Met...,Netral
3,Razia PPKM Petugas Bongkar dan Bakar Kandang M...,positif
4,RT @kecpiyungan1: Instruksi Bupati Nomor 30/In...,positif


In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['Kalimat'], "@[\w]*")

In [ ]:
def remove(tweet):
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    return tweet
df['remove_http'] = df['remove_user'].apply(lambda x: remove(x))
df.sort_values("remove_http", inplace = True)
df.drop_duplicates(subset ="remove_http", keep = 'first', inplace = True)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pip install Sastrawi

     |████████████████████████████████| 209 kB 8.0 MB/s 


In [ ]:
 #import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean
df['tweet_clean'] = df['remove_http'].apply(lambda x: clean_tweets(x))

In [ ]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))

In [ ]:
df.sort_values("Tweet", inplace = True)

In [ ]:
df.drop_duplicates(subset ="Tweet", keep = 'first', inplace = True)

In [ ]:
df.drop(df.columns[[0]], axis = 1, inplace = True)

In [ ]:
df.drop(df.columns[[1,2,3]], axis = 1, inplace = True)

In [ ]:
df

,label,Tweet
33,positif,
863,Negatif,ace boom sale hadir ace cipinang indah mall la...
412,Netral,airlangga provinsi masuk ppkm level jawa-bali ...
526,Positif,airlangga realisasi lindung sosial capai rp tr...
532,Positif,ajar tatap muka mari lindung anak tular covid ...
...,...,...
650,positif,wisata kabupaten malang buka vaksinasi capai p...
600,Netral,xe x xcpenerapan ppkm level terap minggu xe x ...
264,Negatif,xf xf x xe wa xe xc x info org xf xf xc x
123,Netral,yatim piatu korban covid yogyakarta santun rp ...


In [ ]:
from google.colab import files

df = pd.DataFrame(df, columns= ['label', 'Tweet'])
df.to_csv('#PPKM_PRE.csv', index = False, header=True)
files.download('#PPKM_PRE.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>